In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,BatchNormalization,Dropout,Input,Activation,Flatten,Concatenate,Add
from tensorflow.keras.layers.experimental.preprocessing import Rescaling,RandomRotation,RandomZoom,RandomFlip,Resizing
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
import numpy as np 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import re

In [2]:
train_ds, train_info=tfds.load('german_credit_numeric', split='train', with_info=True,shuffle_files=True, as_supervised=True,data_dir='./data')
#train_ds,ds_info  = tfds.load('german_credit_numeric', split='train',data_dir='./data',as_supervised=True, with_info=True)

In [3]:
next(iter(train_ds))

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 3,  6,  4, 13,  2,  5,  1,  4,  3, 28,  3,  2,  2,  2,  1,  1,  0,
         1,  0,  0,  1,  0,  0,  1])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [4]:
df = tfds.as_dataframe(train_ds,train_info)
# for col, dtype in df.dtypes.items():
#     if dtype == np.object:  # Only process byte object columns.
#         df[col] = df[col].apply(lambda x: x.decode("utf-8"))
df.head(5)

,features,label
0,"[3, 6, 4, 13, 2, 5, 1, 4, 3, 28, 3, 2, 2, 2, 1...",1
1,"[4, 4, 2, 6, 1, 2, 2, 3, 1, 23, 3, 1, 2, 1, 1,...",1
2,"[4, 24, 4, 20, 1, 3, 2, 4, 3, 37, 3, 1, 1, 2, ...",1
3,"[4, 18, 2, 11, 5, 2, 2, 2, 1, 21, 3, 1, 1, 2, ...",1
4,"[4, 6, 2, 13, 3, 3, 1, 4, 1, 62, 3, 1, 1, 1, 1...",1


In [5]:
len(df.iloc[0,0])

24

In [6]:
['feature_' + str(x) for x in range(0,len(df.iloc[0,0]))]

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23']

In [7]:
def get_list(x_in):
    print(x_in.tolist()[0])
    return x_in.tolist()[0]

df2=pd.DataFrame([])
df2[['feature_' + str(x) for x in range(0,len(df.iloc[0,0]))]]=df[['features']].apply(lambda y:pd.Series(y[0]),result_type='expand',axis=1)

In [8]:
df2

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23
0,3,6,4,13,2,5,1,4,3,28,...,1,1,0,1,0,0,1,0,0,1
1,4,4,2,6,1,2,2,3,1,23,...,1,0,0,1,0,1,0,0,1,0
2,4,24,4,20,1,3,2,4,3,37,...,1,1,0,1,0,0,1,0,0,1
3,4,18,2,11,5,2,2,2,1,21,...,1,0,0,1,0,1,0,0,0,1
4,4,6,2,13,3,3,1,4,1,62,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,6,2,12,2,5,3,2,2,43,...,1,1,0,1,0,0,1,0,0,1
996,4,36,4,35,1,4,3,4,3,37,...,1,1,0,1,0,0,1,0,0,1
997,4,24,4,26,1,5,4,3,1,46,...,1,0,0,0,1,0,1,0,0,1
998,1,8,4,7,1,5,3,4,1,47,...,1,1,0,1,0,0,1,0,1,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df2,df['label'], test_size=0.20, random_state=42)

In [10]:
inpt=Input(24,name='features')
x1=BatchNormalization()(inpt)

x=Dense(64)(x1)
x=BatchNormalization()(x)
x=Activation(activation='relu')(x)
x=Dropout(0.2)(x)

# x=Dense(32)(x)
# x=BatchNormalization()(x)
# x=Activation(activation='swish')(x)
# x=Dropout(0.2)(x)

x=Concatenate()([x,x1])

x=Dense(1,activation='sigmoid',name='survived')(x)
model=Model(inputs=inpt,outputs=x)
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),
              metrics=[tf.keras.metrics.AUC(num_thresholds=200, curve='ROC'),tf.keras.metrics.PrecisionAtRecall(1, num_thresholds=200, name=None, dtype=None),'accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           [(None, 24)]         0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 24)           96          features[0][0]                   
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           1600        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64)           256         dense[0][0]                      
______________________________________________________________________________________________

In [11]:
X_train_ds=tf.data.Dataset.from_tensor_slices((tf.constant(X_train.astype('float32')),tf.constant(y_train.astype('float32')))).batch(100).prefetch(1)
X_test_ds=tf.data.Dataset.from_tensor_slices((tf.constant(X_test.astype('float32')),tf.constant(y_test.astype('float32')))).batch(100).prefetch(1)

In [12]:
model.fit(X_train_ds,validation_data=X_test_ds,epochs=97)

Epoch 1/97
8/8 [==============================] - 3s 223ms/step - loss: 1.0043 - auc: 0.4982 - precision_at_recall: 0.7006 - accuracy: 0.3783 - val_loss: 3.1951 - val_auc: 0.4803 - val_precision_at_recall: 0.7000 - val_accuracy: 0.3000
Epoch 2/97
8/8 [==============================] - 0s 39ms/step - loss: 1.0058 - auc: 0.4746 - precision_at_recall: 0.7017 - accuracy: 0.3792 - val_loss: 2.0866 - val_auc: 0.4883 - val_precision_at_recall: 0.7000 - val_accuracy: 0.3050
Epoch 3/97
8/8 [==============================] - 0s 17ms/step - loss: 0.9797 - auc: 0.4836 - precision_at_recall: 0.6985 - accuracy: 0.4000 - val_loss: 1.6090 - val_auc: 0.4895 - val_precision_at_recall: 0.7000 - val_accuracy: 0.2850
Epoch 4/97
8/8 [==============================] - 0s 17ms/step - loss: 0.9780 - auc: 0.4998 - precision_at_recall: 0.6998 - accuracy: 0.3959 - val_loss: 1.3430 - val_auc: 0.4886 - val_precision_at_recall: 0.7000 - val_accuracy: 0.2900
Epoch 5/97
8/8 [==============================] - 0s 17ms/s

In [13]:
!py -m pip install shap

In [14]:
import shap

explainer = shap.KernelExpla    iner(model.predict, X_train, link="logit")
shap_values = explainer.shap_values(X_test, nsamples=100)

SyntaxError: invalid syntax (<ipython-input-14-ed8c94c63ede>, line 3)

In [1]:
# plot the SHAP values for the Setosa output of the first instance
shap.initjs()

shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")

NameError: name 'shap' is not defined